In [58]:
import pandas as pd
import numpy as np

In [59]:
# 1.读取数据
column_name = ['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape',
               'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
               'Normal Nucleoli', 'Mitoses', 'Class']
data = pd.read_csv("./cancer/breast-cancer-wisconsin.data", names=column_name)

In [60]:
# 2.缺失值处理
# 1）替换->np.nan
data = data.replace(to_replace="?", value=np.nan)
# 2）删除缺失样本
data.dropna(inplace=True)

In [61]:
data.isnull().any()# 不存在缺失值

Sample code number             False
Clump Thickness                False
Uniformity of Cell Size        False
Uniformity of Cell Shape       False
Marginal Adhesion              False
Single Epithelial Cell Size    False
Bare Nuclei                    False
Bland Chromatin                False
Normal Nucleoli                False
Mitoses                        False
Class                          False
dtype: bool

In [62]:
# 3.划分数据集
from sklearn.model_selection import train_test_split

In [63]:
# 筛选特征值和目标值
x = data.iloc[:, 1:-1]
y = data["Class"]

In [64]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [65]:
# 4.特征工程-标准化
from sklearn.preprocessing import StandardScaler

In [66]:
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [67]:
# 5.预估器流程
from sklearn.linear_model import LogisticRegression

In [68]:
estimator = LogisticRegression()
estimator.fit(x_train, y_train)

LogisticRegression()

In [69]:
# 逻辑回归的模型参数回归系数和偏置
estimator.coef_

array([[ 1.42049062,  0.08450887,  0.87576768,  0.39005224, -0.0402723 ,
         1.34964856,  1.06163256,  0.6374456 ,  0.67836592]])

In [70]:
estimator.intercept_

array([-1.17013845])

In [71]:
# 6.模型调优评估
# 方法一：直接比对真实值
y_predict = estimator.predict(x_test)
print("y_predict:\n", y_predict)
print("直接比对真实值和预测值：\n", y_test == y_predict)

# 方法二：计算准确率
score = estimator.score(x_test, y_test)
print("准确率为：\n", score)

y_predict:
 [4 2 2 4 4 2 4 4 2 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 2 4 2 2 4 2 4 4 2 4 2 4 2
 4 4 2 2 2 2 2 2 2 4 4 2 4 2 2 4 2 2 2 2 2 4 2 2 4 4 2 2 4 2 4 2 2 2 2 4 4
 4 4 2 2 4 2 4 2 4 2 4 2 2 2 4 2 2 2 2 2 2 2 2 4 2 2 2 2 2 4 2 2 4 4 4 2 2
 2 4 2 4 4 2 2 4 4 2 2 4 2 2 2 2 2 2 2 2 2 4 2 2 2 2 4 2 4 2 2 4 2 2 2 2 4
 2 2 2 2 4 2 4 2 2 4 4 2 2 2 2 4 2 2 2 2 4 2 4]
直接比对真实值和预测值：
 226    True
170    True
468    True
68     True
604    True
       ... 
384    True
543    True
283    True
244    True
336    True
Name: Class, Length: 171, dtype: bool
准确率为：
 0.9649122807017544


In [72]:
# 查看精确率，召回率，F1—score
from sklearn.metrics import classification_report

In [73]:
report = classification_report(y_test,y_predict,labels=[2,4],target_names=["良心","恶性"])

In [75]:
print(report)

              precision    recall  f1-score   support

          良心       0.96      0.99      0.97       112
          恶性       0.98      0.92      0.95        59

    accuracy                           0.96       171
   macro avg       0.97      0.95      0.96       171
weighted avg       0.97      0.96      0.96       171



In [76]:
# y_true:每个样本的真实类别，必须为0(反例),1(正例)标记
# 将y_test转换成0，1
y_true = np.where(y_test > 3, 1, 0)

In [77]:
y_true

array([1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1])

In [78]:
# 映入AUC分类评价指标 
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_predict)

0.9531628329297821